In [1]:
import time
from multiprocessing import Pool
import functools

In [6]:
import requests
query="The+Moving+Finger+Agatha+Christie"
ids="9CkX1DBb0P8C"
urlstring='https://www.googleapis.com/books/v1/volumes?q='+query+'&fields=items&maxResults=10&id='+ids
#urlstring="https://www.googleapis.com/books/v1/volumes?q=carol+marinelli&maxResults=10&id=9CkX1DBb0P8C"
resp = requests.get(urlstring)
if resp.status_code != 200:
    # This means something went wrong.
    print(resp.status_code)

In [7]:
response=resp.json()

In [8]:
for mm in response[u'items']:
    dictt=mm[u'volumeInfo']
    
    print dictt[u'authors'],dictt[u'title']

[u'Agatha Christie'] The Moving Finger (Miss Marple) [u'Agatha Christie'] The Moving Finger [u'Agatha Christie'] The Moving Finger [u'Agatha Christie'] The Moving Finger [u'James Zemboy'] The Detective Novels of Agatha Christie [u'Edith Wharton'] The Moving Finger [u'Agatha Christie'] Collins the Moving Finger (ELT Reader) [u'Agatha Christie'] Miss Marple 3-Book Collection 1: The Murder at the Vicarage, The Body in the Library, The Moving Finger [u'Agatha Christie'] The Hollow [u'Agatha Christie'] Miss Marple Omnibus


In [26]:
type(response)

dict

In [19]:
##get book cover
import urllib
imgurl=response[u'items'][0][u'volumeInfo'][u'imageLinks'][u'thumbnail'].encode('ascii','ignore')
imgname="/home/some/img.jpg"
urllib.urlretrieve(imgurl,imgname)

('/home/some/img.jpg', <httplib.HTTPMessage instance at 0x7fcc54164830>)

In [47]:
d="/media/some/sophie-hhd/transmitt/fic/#Calibre Fiction/"
dd="/media/some/sophie-hhd/transmitt/fic/Fictional Literature/"
files=! find "$d" "$dd" -type f -iname "*.epub"
len(files)

8837

In [60]:
#test
f=files[-4053]
#print f
s=getFileGoogleMeta(f,0)

The+Inferno+Dante+Alighieri


In [54]:
titles={}#load_obj('data/titlesISBN')#{}
keys=[]#load_obj('data/titleskeysISBN')#[]

In [55]:
flist=list(filter(lambda x: x[x.rindex('/')+1:x.rindex('.')] not in keys, files))
len(flist),len(keys),len(files)

(8837, 0, 8837)

In [57]:

t0 = time.time()
books=[]
cnt=0
p=Pool(10)
try:
    books=p.map(getFileIsbnMeta, flist[20:30])
except:
    print "errror" 
    p.terminate()
    p.close()
#for f in flist[10:20]:
#    books.append(getFileGoogleMeta(f))
#    time.sleep(10) 

p.terminate()
p.close()
for book in books:
    try:
        if book[0] not in keys and len(book)>2:
            titles[book[0]]=book[1:]
    except:
        cnt+=1
print "exception count ",cnt        
print len(list(titles.keys()))
save_obj(titles, 'data/titlesISBN')
keys=list(titles.keys())
save_obj(keys,'data/titleskeysISBN')
print 'finish parallel'

t1=time.time()
total = t1-t0
print('time: ',total)

Djinn+in+the+Nightingales+Eye+A+S+Byatt
A+S+Byatt+Djinn+in+the+Nightingales+Eye
Elementals+A+S+Byatt
A+S+Byatt+Elementals
Little+Black+Book+of+Stories+A+S+Byatt
A+S+Byatt+Little+Black+Book+of+Stories
Possession+A+S+Byatt
A+S+Byatt+Possession
Ragnarok+A+S+Byatt
A+S+Byatt+Ragnarok
Ragnarok+A+S+Byatt
A+S+Byatt+Ragnarok
Sugar+and+Other+Stories+A+S+Byatt
A+S+Byatt+Sugar+and+Other+Stories
The+Biographers+Tale+A+S+Byatt
A+S+Byatt+The+Biographers+Tale
The+Childrens+Book+A+S+Byatt
A+S+Byatt+The+Childrens+Book
The+Game+A+S+Byatt
A+S+Byatt+The+Game
exception count  0
8
finish parallel
('time: ', 195.14910888671875)


In [23]:
f='/media/some/sophie-hhd/transmitt/fic/Fictional Li4terature/True Crime Collection/Portrait of a Killer_ Jack the Ripper Case Closed - Patricia Cornwell/Patricia Cornwell - Portrait of a Killer_ Jack the_sed (v5.0).epub'

In [59]:
m=getFileGoogleMeta(f)

The+Game+A+S+Byatt


In [58]:
import os
import string
import comFuncs
from comFuncs import *
import requests
import os
import string
import time
def getGoogleMeta(title,author):
    query=title+" "+author
    #remove none alphanum characters
    query=''.join(q for q in query if q in string.letters+string.digits+" "+"_")
    #remove extra white space
    query=' '.join(query.split())
    #replace white space with +
    query='+'.join(query.split())
    #print query
    ids="9CkX1DBb0P8C"
    urlstring='https://www.googleapis.com/books/v1/volumes?q='+query+'&fields=items&maxResults=10&id='+ids
#urlstring="https://www.googleapis.com/books/v1/volumes?q=carol+marinelli&maxResults=10&id=9CkX1DBb0P8C"
    cnt=0
    while True:
        resp = requests.get(urlstring)
        if resp.status_code == 200:
            break
        cnt+=1
        time.sleep(1)
    data=resp.json()
    #print data
    if type(data)==dict and u'items' in data.keys():
        return data[u'items']
    else:
        return ""
        #print "service unavailable"
def getFileGoogleMeta(f,smscr=1.0):
    #print "start"
    f=f.encode('ascii','ignore')
    fp,ff= os.path.split(f)
    fn, fext = os.path.splitext(ff)
    fn=re.sub(" \(v[0-9].[0-9]\)","",fn)
    #print fn
    seri=''
    sm=0
    mtd=None
    sp=fn.split()
    if len(sp)>4:
        tit=sp[-2:]
        auth=sp[:-2]
    elif len(sp) >1:
        tit=fn.split()[-1:]
        auth=fn.split()[:-1]
    else:
        return "no meta"
    
    if fn.count(' - ')==1:
        tit,auth=fn.split(' - ')
    elif fn.count(' - ')==2:
        seri,tit,auth=fn.split(' - ')
        
    #print "title",tit,"author",auth
   
    if tit.count('_ ')>0:
        spp=tit.split('_')[0]
        #print spp
        if spp.count(' ')>0:
            tit=' '.join(spp.split())
        else:
            tit=spp
    if auth.count('_ ')>0:
        spp=auth.split('_')[0]
        #print spp
        if spp.count(' ')>0:
            auth=' '.join(spp.split())
        else:
            auth=spp
    if tit.count(' ')>0:
        title=''.join(l for l in ' '.join(tit.split()) if l in string.printable)
    else:
        title=''.join(l for l in tit if l in string.printable)
    if auth.count(" ")>0:    
        author=''.join(l for l in ' '.join(auth.split()) if l in string.printable)
    else:
        author=''.join(l for l in auth if l in string.ascii_letters+" .")
    #print tit,auth
    #print title,'..... ....',author
  
    metad =getGooleMeta(title,author)
    if metad !="" and type(metad)==list: 
        for mdd in metad:
            if u'volumeInfo' in mdd.keys():
                md=mdd[u'volumeInfo']
                if type(md)==dict and u"authors" in md.keys() and u"title" in md.keys():
                    if len(md[u'authors'])>0:
                        authsim=similarity(md[u'authors'][0].encode('ascii','ignore'),str(author))
                    else:
                        authsim=0
                    titsim=similarity(md[u'title'].encode('ascii','ignore'),str(title))
                    #print titsim,authsim
                    if titsim+authsim >sm:
                        sm=titsim+authsim
                        mtd=md
        if sm >smscr and mtd !=None:
            return fn,fp,mtd,sm
    #if no data found switch title and author
    metad =getGoogleMeta(author,title)
    if metad !="" and type(metad)==list: 
        for mdd in metad:
            if u'volumeInfo' in mdd.keys():
                md=mdd[u'volumeInfo']
                if type(md)==dict and u"authors" in md.keys() and u"title" in md.keys():
                    if len(md[u'authors'])>0:
                        authsim=similarity(md[u'authors'][0].encode('ascii','ignore'),str(title))
                    else:
                        authsim=0
                    titsim=similarity(md[u'title'].encode('ascii','ignore'),str(author))
                    #print titsim,authsim
                    if titsim+authsim >sm:
                        sm=titsim+authsim
                        mtd=md
        if sm >smscr and mtd !=None:
            return fn,fp,mtd,sm
    # if no data found try seri, auth ,title  combinaton
    if seri !='':
        metad =getGoogleMeta(title,str(seri))
        if metad !="" and type(metad)==list:
            
            for mdd in metad:
                if u'volumeInfo' in mdd.keys():
                    md=mdd[u'volumeInfo']
                    if type(md)==dict and u"authors" in md.keys() and u"title" in md.keys():
                        if len(md[u'authors'])>0:
                            authsim=similarity(md[u'authors'][0].encode('ascii','ignore'),str(seri))
                        else:
                            authsim=0
                        titsim=similarity(md[u'title'].encode('ascii','ignore'),str(title))
                        #print titsim,authsim
                        if titsim+authsim >sm:
                            sm=titsim+authsim
                            mtd=md
            if sm >smscr and mtd !=None:
                return fn,fp,mtd,sm
        metad =getGoogleMeta(seri,str(author))
        if metad !="" and type(metad)==list: 
            for mdd in metad:
                if u'volumeInfo' in mdd.keys():
                    md=mdd[u'volumeInfo']
                    if type(md)==dict and u"authors" in md.keys() and u"title" in md.keys():
                        if len(md[u'authors'])>0:
                            authsim=similarity(md[u'authors'][0].encode('ascii','ignore'),str(author))
                        else:
                            authsim=0
                        titsim=similarity(md[u'title'].encode('ascii','ignore'),str(seri))
                        #print titsim,authsim
                        if titsim+authsim >sm:
                            sm=titsim+authsim
                            mtd=md    
            if sm >smscr and mtd !=None:
                return fn,fp,mtd,sm
        metad =getGoogleMeta(seri,str(title))
        if metad !="" and type(metad)==list: 
            for mdd in metad:
                if u'volumeInfo' in mdd.keys():
                    md=mdd[u'volumeInfo']
                    if type(md)==dict and u"authors" in md.keys() and u"title" in md.keys():
                        if len(md[u'authors'])>0:
                            authsim=similarity(md[u'authors'][0].encode('ascii','ignore'),str(title))
                        else:
                            authsim=0
                        titsim=similarity(md[u'title'].encode('ascii','ignore'),str(seri))
                        #print titsim,authsim
                        if titsim+authsim >sm:
                            sm=titsim+authsim
                            mtd=md
            if sm >smscr and mtd !=None:
                return fn,fp,mtd,sm
        metad =getGoogleMeta(author,str(seri))
        if metad !="" and type(metad)==list: 
            for mdd in metad:
                if u'volumeInfo' in mdd.keys():
                    md=mdd[u'volumeInfo']
                    if type(md)==dict and u"authors" in md.keys() and u"title" in md.keys():
                        if len(md[u'authors'])>0:
                            authsim=similarity(md[u'authors'][0].encode('ascii','ignore'),str(seri))
                        else:
                            authsim=0
                        titsim=similarity(md[u'title'].encode('ascii','ignore'),str(author))
                    #print titsim,authsim
                        if titsim+authsim >sm:
                            sm=titsim+authsim
                            mtd=md               
        if sm >smscr and mtd !=None:
            return fn,fp,mtd,sm
    return "NO meta"   
    